In [ ]:
using ITensors, ITensorMPS

# Creating function to compute the connected correlation for X
function connected_correlation_X(psi, sites, m, n)

    # Create MPO for Xm
    osXm = OpSum()
    osXm += "X", m
    Xm = MPO(osXm, sites)

    # Create MPO for Xn
    osXn = OpSum()
    osXn += "X", n
    Xn = MPO(osXn, sites)

    # Create MPO for XmXn
    osXmn = OpSum()
    osXmn += "X", m, "X", n
    XmXn = MPO(osXmn, sites)

    # Expectation values
    X_m = inner(psi', Xm, psi) # <psi|Xm|psi>
    X_n = inner(psi', Xn, psi) # <psi|Xn|psi>
    X_mX_n = inner(psi', XmXn, psi) # <psi|XmXn|psi>

    return X_mX_n - X_m * X_n # <XmXn> - <Xm><Xn> 
end

# Creating function to compute the connected correlation for Z
function connected_correlation_Z(psi, sites, m, n)

    # Create MPO for Zm
    osZm = OpSum()
    osZm += "Z", m
    Zm = MPO(osZm, sites)

    # Create MPO for Zn
    osZn = OpSum()
    osZn += "Z", n
    Zn = MPO(osZn, sites)

    # Create MPO for ZmZn
    osZmn = OpSum()
    osZmn += "Z", m, "Z", n
    ZmZn = MPO(osZmn, sites)

    # Expectation values
    Z_m = inner(psi', Zm, psi) # <psi|Zm|psi>
    Z_n = inner(psi', Zn, psi) # <psi|Zn|psi>
    Z_mZ_n = inner(psi', ZmZn, psi) # <psi|ZmZn|psi>

    return Z_mZ_n - Z_m * Z_n # <ZmZn> - <Zm><Zn>
end


let
    # Create spin-1/2 indices
    L = 30
    sites = siteinds("S=1/2", L)

    # Create the Hamiltonian of the Ising chain at its self-dual critical point
    # from the O'Brien and Fendley paper.
    os_I = OpSum()
    for j in 1:(L - 1)
        os_I -= "X", j 
        os_I -= "Z", j, "Z", j + 1
    end
    # Boundary conditions
    os_I -= "X", L # j = L
    os_I -= "Z", L, "Z", 1

    H_I = MPO(os_I, sites)

    # Create the Hamiltonian of the three-spin interaction from the O'Brien and Fendley paper.
    os_3 = OpSum()
    for j in 1:(L - 2)
        os_3 += "X", j, "Z", j + 1, "Z", j + 2
        os_3 += "Z", j, "Z", j + 1, "X", j + 2
    end
    os_3 += "X", L - 1, "Z", L, "Z", 1
    os_3 += "X", L, "Z", 1, "Z", 2
    os_3 += "Z", L - 1, "Z", L, "X", 1
    os_3 += "Z", L, "Z", 1, "X", 2
    
    H_3 = MPO(os_3, sites)

    # Ground state coupling coefficients
    # lambda_I = 1
    # lambda_3 = 1

    # Tri-Critical Ising coupling coefficients
    lambda_I = 0.7597
    lambda_3 = 0.6503
    
    H = 2 * lambda_I * H_I + lambda_3 * H_3

    # Create an initial random matrix product state
    psi0 = random_mps(sites)

    # Sweeps, maximum dimesnions, cutoff value, and observer
    nsweeps = 20
    maxdim = [10, 25, 50, 100, 250, 500, 1000, 2500, 5000]
    cutoff = 1.0e-7
    observer = DMRGObserver(; energy_tol=1.0e-7, minsweeps=4)

    energy1, psi1 = dmrg(H, psi0; nsweeps, maxdim, cutoff, observer=observer)
    energy2, psi2 = dmrg(H, [psi1], psi0; nsweeps, maxdim, cutoff, weight = 100, observer=observer)
    energy3, psi3 = dmrg(H, [psi1, psi2], psi0; nsweeps, maxdim, cutoff, weight = 100, observer=observer)
    energy4, psi4 = dmrg(H, [psi1, psi2, psi3], psi0; nsweeps, maxdim, cutoff, weight = 100, observer=observer)
    
    m = 1
    n = L

    CCX0 = connected_correlation_X(psi0, sites, m, n)
    CCX1 = connected_correlation_X(psi1, sites, m, n)
    CCX2 = connected_correlation_X(psi2, sites, m, n)
    CCX3 = connected_correlation_X(psi3, sites, m, n)
    CCX4 = connected_correlation_X(psi4, sites, m, n)

    CCZ0 = connected_correlation_Z(psi0, sites, m, n)
    CCZ1 = connected_correlation_Z(psi1, sites, m, n)
    CCZ2 = connected_correlation_Z(psi2, sites, m, n)
    CCZ3 = connected_correlation_Z(psi3, sites, m, n)
    CCZ4 = connected_correlation_Z(psi4, sites, m, n)

    println("Connected correlation X for state 0 = $CCX0")
    println("Connected correlation X for state 1 = $CCX1")
    println("Connected correlation X for state 2 = $CCX2")
    println("Connected correlation X for state 3 = $CCX3")
    println("Connected correlation X for state 4 = $CCX4")

    println("Connected correlation Z for state 0 = $CCZ0")
    println("Connected correlation Z for state 1 = $CCZ1")
    println("Connected correlation Z for state 2 = $CCZ2")
    println("Connected correlation Z for state 3 = $CCZ3")
    println("Connected correlation Z for state 4 = $CCZ4")   

    nothing
end



After sweep 1 energy=-45.61124024993068  maxlinkdim=4 maxerr=1.25E-15 time=0.081
After sweep 2 energy=-46.046006239822006  maxlinkdim=16 maxerr=4.36E-12 time=0.063
After sweep 3 energy=-46.10085838140603  maxlinkdim=50 maxerr=1.62E-10 time=0.304
After sweep 4 energy=-46.130063855183785  maxlinkdim=100 maxerr=1.42E-09 time=1.979
After sweep 5 energy=-46.146455918389336  maxlinkdim=182 maxerr=9.97E-11 time=5.233
After sweep 6 energy=-46.151525348602384  maxlinkdim=198 maxerr=9.99E-11 time=6.940
After sweep 7 energy=-46.15238448674136  maxlinkdim=199 maxerr=9.99E-11 time=7.062
After sweep 8 energy=-46.15249513927195  maxlinkdim=192 maxerr=9.99E-11 time=6.840
After sweep 9 energy=-46.15250810614014  maxlinkdim=185 maxerr=9.98E-11 time=6.653
After sweep 10 energy=-46.152510077488536  maxlinkdim=196 maxerr=9.93E-11 time=6.461
After sweep 11 energy=-46.152510277488005  maxlinkdim=179 maxerr=9.99E-11 time=7.365
After sweep 12 energy=-46.15251030052937  maxlinkdim=174 maxerr=9.98E-11 time=6.302